In [1]:
import pandas as pd
import numpy as np
import pickle
import sklearn
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv("mon_standard_dataset.csv")

In [3]:
data = data[['Direction_Size_Sequence', 'Label']]

In [4]:
data['Direction_Sequence'] = data['Direction_Size_Sequence'].apply(lambda x: [1 if i > 0 else -1 for i in eval(x)])

In [5]:
# 라벨이 0~94 사이인 데이터만 필터링
data = data[data['Label'].between(0, 94)]

In [6]:
def adjust_sequence_length(sequence, target_length=10000, padding_value=-1):
    if isinstance(sequence, (int, float, np.float32, np.float64)):
        sequence = [sequence]
    if len(sequence) < target_length:
        sequence = sequence + [padding_value] * (target_length - len(sequence))
    else:
        sequence = sequence[:target_length]
    return sequence


In [7]:
# Convert DataFrame column to NumPy array
sequence_array = data['Direction_Sequence'].to_numpy()

# Adjust sequence lengths
sequence_array = np.array([adjust_sequence_length(seq) for seq in sequence_array])

sequence_array.shape

(19000, 10000)

In [8]:
data['Label'].shape

(19000,)

In [9]:
mon_data = pd.DataFrame({
    'Direction_Sequence': list(sequence_array),
    'Label': data['Label'].values 
})

In [10]:
# Save the processed data
# pandas에서 .to_pickle()을 사용하여 DataFrame을 저장하고, pd.read_pickle()을 사용하여 불러오기
mon_data.to_pickle("mon_data.pkl")

print("Data processed and saved successfully.")

Data processed and saved successfully.


### 데이터 처리 확인


In [11]:
mon_data.tail()

,Direction_Sequence,Label
18995,"[-1, -1, 1, -1, 1, -1, 1, 1, -1, -1, -1, -1, -...",94
18996,"[-1, -1, 1, -1, 1, -1, 1, 1, -1, -1, -1, -1, -...",94
18997,"[-1, -1, 1, -1, 1, -1, 1, 1, -1, -1, -1, -1, -...",94
18998,"[-1, -1, 1, -1, 1, -1, 1, 1, -1, -1, -1, -1, -...",94
18999,"[-1, -1, 1, -1, 1, -1, 1, 1, -1, -1, -1, -1, -...",94


In [14]:
# padding 확인, 2525 대신 다른 데이터 넣어보기
len(mon_data.loc[2520, 'Direction_Sequence'])

10000

In [13]:
# 라벨별 데이터 모아보기
# 95 * 200 = 19000 
Label_10 = mon_data[mon_data['Label']==94]
Label_10.shape

(200, 2)

In [15]:
def split_data(data, m, n):
    """
    데이터프레임에서 클래스 라벨이 m부터 n까지인 데이터를 추출합니다.
    
    Parameters:
    data (pd.DataFrame): 원본 데이터프레임
    m (int): 시작 클래스 라벨
    n (int): 끝 클래스 라벨
    
    Returns:
    pd.DataFrame: 추출된 데이터
    """
    return data[data['Label'].between(m, n)]

In [16]:
# 사용 예시
m, n = 0, 10  # 클래스 라벨 0부터 10까지 추출
subset_data = split_data(mon_data, m, n)
print(f"추출된 데이터 shape: {subset_data.shape}")
print(f"추출된 데이터의 라벨 분포:\n{subset_data['Label'].value_counts().sort_index()}")

추출된 데이터 shape: (2200, 2)
추출된 데이터의 라벨 분포:
Label
0     200
1     200
2     200
3     200
4     200
5     200
6     200
7     200
8     200
9     200
10    200
Name: count, dtype: int64


In [19]:
def accumulate_data(existing_data, m, n):
    new_data = mon_data[mon_data['Label'].between(m, n)]
    combined_data = pd.concat([existing_data, new_data], axis=0, ignore_index=True)
    
    return combined_data


In [20]:
# 초기 데이터셋 (클래스 0~44)
existing_data = mon_data[mon_data['Label'].between(0, 44)]

# 클래스 45~49 데이터 추가
existing_data = accumulate_data(existing_data, 45, 49)

# 결과 확인
print(f"최종 데이터 shape: {existing_data.shape}")
print(f"클래스 분포:\n{existing_data['Label'].value_counts().sort_index()}")


최종 데이터 shape: (10000, 2)
클래스 분포:
Label
0     200
1     200
2     200
3     200
4     200
5     200
6     200
7     200
8     200
9     200
10    200
11    200
12    200
13    200
14    200
15    200
16    200
17    200
18    200
19    200
20    200
21    200
22    200
23    200
24    200
25    200
26    200
27    200
28    200
29    200
30    200
31    200
32    200
33    200
34    200
35    200
36    200
37    200
38    200
39    200
40    200
41    200
42    200
43    200
44    200
45    200
46    200
47    200
48    200
49    200
Name: count, dtype: int64


In [21]:
import pandas as pd

def accumulate_data2(data1, data2):
    """
    두 데이터프레임을 세로로 합칩니다.

    Parameters:
    data1 (pd.DataFrame): 첫 번째 데이터프레임
    data2 (pd.DataFrame): 두 번째 데이터프레임

    Returns:
    pd.DataFrame: 합쳐진 데이터프레임
    """
    combined_data = pd.concat([data1, data2], axis=0, ignore_index=True)
    return combined_data

In [22]:
# 테스트 코드
# mon_data에서 0~44 클래스와 45~49 클래스 데이터 추출
data1 = mon_data[mon_data['Label'].between(0, 44)]
data2 = mon_data[mon_data['Label'].between(45, 49)]

# accumulate_data2 함수 실행
result = accumulate_data2(data1, data2)

# 결과 확인
print(f"합쳐진 데이터의 shape: {result.shape}")
print(f"원본 데이터1의 shape: {data1.shape}")
print(f"원본 데이터2의 shape: {data2.shape}")
print(f"합쳐진 데이터의 Label 범위: {result['Label'].min()} ~ {result['Label'].max()}")

# 데이터가 올바르게 합쳐졌는지 확인
assert len(result) == len(data1) + len(data2), "합쳐진 데이터의 길이가 올바르지 않습니다."
assert list(result.columns) == list(mon_data.columns), "컬럼이 올바르지 않습니다."
assert result['Label'].min() == 0 and result['Label'].max() == 49, "Label 범위가 올바르지 않습니다."

print("테스트가 성공적으로 완료되었습니다.")


합쳐진 데이터의 shape: (10000, 2)
원본 데이터1의 shape: (9000, 2)
원본 데이터2의 shape: (1000, 2)
합쳐진 데이터의 Label 범위: 0 ~ 49
테스트가 성공적으로 완료되었습니다.
